In [2]:
import random
import numpy as np
from datetime import datetime, timedelta
import pandas as pd

df = pd.read_csv("hist_routing_orders.csv")
start_date = datetime(2022,2,1,0,0)
start_tick = 480
colors = ['blue', 'darkblue', 'purple', 'black']
oncall_order_color = 'green'
terminated_order_color = "red"
delayed_order_color = 'orange'
last_tick = 1439

render_route_num = 3

station_loc = [32.72329226, -117.0718922]

def get_stats(tick, is_last=False):
    if is_last:
        tick = last_tick
    max_tick = df.loc[((df['tick'] <= tick)), 'tick'].max()
    df_stat = df[((df['tick'] == max_tick))]
    order_status_stat = df_stat[['order_id', 'order_status']].groupby('order_status').count().reset_index()
    order_status_stat = {row['order_status']: row['order_id'] for _, row in order_status_stat.iterrows()}
    info = ""
    for key, val in order_status_stat.items():
        key_str = ("RTB" if key == 'Dummy' else key)
        info += f"{key_str}: {val}, "
    return info

def create_geojson_features(_route_name=None):
    if _route_name:
        route_name_list = [_route_name]
    else:
        df_oncall_cnt = df.loc[((df['tick'] == last_tick) & (df['type'] == "ONCALL")), ['route_name', 'order_id']].groupby('route_name').count().reset_index()
        df_oncall_cnt = df_oncall_cnt[df_oncall_cnt['order_id'] >= 3]
        route_name_list = np.random.choice(df_oncall_cnt['route_name'].unique(), size=render_route_num, replace=False)
    print('> Creating GeoJSON features...')
    features = []
    for i, route_name in enumerate(route_name_list):
        df_route = df[df['route_name'] == route_name]
        df_route_tick = df_route[df_route['tick'] == last_tick]
        df_route_tick.sort_values(by='finish_tick', inplace=True)
        points = [[station_loc[1], station_loc[0]]]
        ticks_to_visit = [(start_date+timedelta(minutes=start_tick)).__str__()]
        route_color = colors[i%len(colors)]
        for _, row in df_route_tick.iterrows():
            if row['type'] != 'PLANNED':
                if row['order_status'] == "Delayed":
                    color = delayed_order_color
                elif row['order_status'] == 'Terminated':
                    color = terminated_order_color
                else:
                    color = oncall_order_color
            else:
                color = route_color
            if row['order_status'] in ['Complete', "Dummy"]:
                points.append([row['order_lng'], row['order_lat']])
                ticks_to_visit.append((start_date+timedelta(minutes=row['finish_tick'])).__str__())
            
            delivery = ("deliver" if row['is_delivery'] else "pickup")
            popup = f"{delivery}, {row['order_status']}"
            feature = {
                'type': 'Feature',
                'geometry': {
                    'type':'Point', 
                    'coordinates':[row['order_lng'],row['order_lat']]
                },
                'properties': {
                    'time': (start_date+timedelta(minutes=row['finish_tick'])).__str__(),
                    'style': {'color' : color},
                    'icon': 'circle',
                    "popup": popup,
                    'iconstyle':{
                        'fillColor': color,
                        'fillOpacity': 0.3,
                        'stroke': 'false',
                        'radius': (5 if row['type']=='ONCALL' else 3)
                    }
                }
            }
            features.append(feature)
        finish_tick_list = df_route_tick['finish_tick'].unique().tolist()
        for i, finished_tick in enumerate(finish_tick_list):
            is_last = (True if (i == len(finish_tick_list) - 1) else False)
            finished_tick = int(finished_tick)
            stat_info = get_stats(finished_tick, is_last=is_last)
            feature = {
                'type': 'Feature',
                'geometry': {
                    'type':'Point', 
                    'coordinates':[station_loc[1], station_loc[0]]
                },
                'properties': {
                    'time': (start_date+timedelta(minutes=finished_tick)).__str__(),
                    'style': {'color' : "green"},
                    "popup": stat_info,
                    'icon': 'circle',
                    'iconstyle':{
                        'fillColor': "green",
                        'fillOpacity': (1.0 if i == 0 else 0.0),
                        'stroke': 'false',
                        'radius': 10,
                    }
                }
            }
            features.append(feature)
        features.append(
            {
                "type": "Feature",
                "geometry": {
                    "type": "LineString",
                    "coordinates": points,
                },
                "properties": {
                    "times": ticks_to_visit,
                    "style": {
                        "color": route_color,
                        "weight": 3,
                    },
                    'iconstyle':{
                        'fillColor': route_color,
                        'fillOpacity': 0.5,
                        'stroke': 'true',
                        'radius': 7
                }
                },
            }
        )
    return features

import folium
from folium.plugins import TimestampedGeoJson

def make_map(features):
    print('> Making map...')
    routing_map = folium.Map(location=station_loc, control_scale=True, zoom_start=12)
    TimestampedGeoJson(
        {'type': 'FeatureCollection',
        'features': features}
        , period='PT1M'
        , add_last_point=True
        , auto_play=False
        , loop=False
        , max_speed=1
        , loop_button=True
        , date_options='YYYY-MM-DD HH:mm'
        , time_slider_drag_update=True
    ).add_to(routing_map)
    print('> Done.')
    return routing_map

features = create_geojson_features()
routing_map = make_map(features)
routing_map.save("routing.html")

> Creating GeoJSON features...


/data/songlei/anaconda3/envs/maroenv/lib/python3.7/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


> Making map...
> Done.
